# Week 2 Day 4 – MobileNetV2 INT8 Quantization (Student Version)

**Goal for today:**

- Load a pretrained **MobileNetV2** model in PyTorch (FP32).
- Measure **model size** and **inference latency** on CPU.
- Load a **quantized INT8 MobileNetV2**.
- Compare FP32 vs INT8 in a simple results table.
- Reflect on what quantization does and why it matters for Edge AI.


## 1. Setup & Imports

If you're on the Raspberry Pi for the first time with PyTorch, you may need to install `torch` and `torchvision` first (this can take a while and is usually done once).

> 💡 **Note:** If `torchvision` is already installed, you can skip any install commands.


In [1]:
# If needed, uncomment and run this cell ONCE to install PyTorch/torchvision on Pi.
# On Pi, you typically use the prebuilt wheels from PyTorch's site.
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

import os
import time
import psutil

import torch
import torch.nn as nn
from torchvision import models

print("PyTorch version:", torch.__version__)
print("CUDA available?:", torch.cuda.is_available())  # Should be False on Pi (CPU-only)

PyTorch version: 2.9.1+cpu
CUDA available?: False


## 2. Device & Helper Functions

We will force everything to run on **CPU**, since the Raspberry Pi 5 is a CPU-based edge device (plus your Hailo accelerator later).

We'll also define a small helper to measure:

- **Latency (ms)** over several runs
- **Model size (MB)** when saved to disk


In [2]:
device = torch.device("cpu")
print("Using device:", device)

def measure_latency(model: nn.Module, example_input: torch.Tensor, 
                    n_warmup: int = 10, n_iters: int = 50) -> float:
    """Measure average inference latency in milliseconds."""
    model.eval()
    model.to(device)
    example_input = example_input.to(device)
    
    # Warmup (not measured)
    with torch.no_grad():
        for _ in range(n_warmup):
            _ = model(example_input)
    
    # Timed runs
    start = time.perf_counter()
    with torch.no_grad():
        for _ in range(n_iters):
            _ = model(example_input)
    end = time.perf_counter()
    
    avg_latency_ms = (end - start) * 1000.0 / n_iters
    return avg_latency_ms


def get_model_size_mb(model: nn.Module, filename: str = "temp_model.pth") -> float:
    """Save model to disk and report file size in MB."""
    torch.save(model.state_dict(), filename)
    size_mb = os.path.getsize(filename) / (1024 * 1024)
    # Clean up file if you like
    # os.remove(filename)
    return size_mb

Using device: cpu


## 3. Load Pretrained FP32 MobileNetV2

We start with the **standard FP32 MobileNetV2**.

> 📦 This will download pretrained weights the first time you run it (requires internet).  
> After that, it will load from cache.


In [3]:
# Newer torchvision versions use arguments like:
# models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
# To stay compatible with more versions, we try both.

try:
    fp32_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
    print("Loaded MobileNetV2 with explicit weights enum.")
except Exception as e:
    print("Falling back to older API:", e)
    fp32_model = models.mobilenet_v2(pretrained=True)

fp32_model.eval()
fp32_model.to(device)

# Create a dummy input tensor: 1 image, 3 channels, 224x224
example_input = torch.randn(1, 3, 224, 224)

# Quick shape check
with torch.no_grad():
    out = fp32_model(example_input.to(device))
print("Output shape:", out.shape)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /home/tokyog/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 27.0MB/s]


Loaded MobileNetV2 with explicit weights enum.
Output shape: torch.Size([1, 1000])


## 4. Benchmark FP32 MobileNetV2

Here we measure:

- **Model size (MB)**  
- **Average latency (ms)** on CPU  
- **RSS Memory usage (MB)** for the current Python process (rough estimate)


In [4]:
process = psutil.Process(os.getpid())

fp32_size_mb = get_model_size_mb(fp32_model, filename="mobilenet_v2_fp32.pth")
fp32_latency_ms = measure_latency(fp32_model, example_input, n_warmup=5, n_iters=20)

fp32_rss_mb = process.memory_info().rss / (1024 * 1024)

print(f"FP32 MobileNetV2 size: {fp32_size_mb:.2f} MB")
print(f"FP32 MobileNetV2 avg latency: {fp32_latency_ms:.2f} ms")
print(f"Approx. RSS memory (process): {fp32_rss_mb:.2f} MB")

FP32 MobileNetV2 size: 13.60 MB
FP32 MobileNetV2 avg latency: 81.83 ms
Approx. RSS memory (process): 464.80 MB


### ✏️ Exercise: Record Your FP32 Numbers

Write down your results:

- FP32 model size (MB):
- FP32 latency (ms):
- FP32 RSS memory (MB):

You'll compare them with the quantized model next.


## 5. Load Quantized INT8 MobileNetV2

TorchVision provides a **quantized version of MobileNetV2** that has already been prepared and quantized using `torch.quantization` under the hood.

This is a **post-training static quantization** example.

> ⚠️ If this import fails, your `torchvision` might not support quantization models.  
> In that case, ask your instructor or check the PyTorch/torchvision version.


In [5]:
# Quantized models live in a separate module in many torchvision versions
try:
    from torchvision.models.quantization import mobilenet_v2 as q_mobilenet_v2

    quantized_model = q_mobilenet_v2(pretrained=True, quantize=True)
    print("Loaded quantized INT8 MobileNetV2 from torchvision.models.quantization.")
    
    quantized_model.eval()
    quantized_model.to(device)
    
    # Quick forward to verify
    with torch.no_grad():
        out_q = quantized_model(example_input.to(device))
    print("Quantized output shape:", out_q.shape)
except Exception as e:
    print("❌ Could not import quantized MobileNetV2:", e)
    print("You may need a newer torchvision, or a different approach to quantization.")

/home/tokyog/edge_bootcamp/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tokyog/edge_bootcamp/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_QuantizedWeights.IMAGENET1K_QNNPACK_V1`. You can also use `weights=MobileNet_V2_QuantizedWeights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/tokyog/edge_bootcamp/lib/python3.11/site-packages/torch/ao/quantization/utils.py:435: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


Downloading: "https://download.pytorch.org/models/quantized/mobilenet_v2_qnnpack_37f702c5.pth" to /home/tokyog/.cache/torch/hub/checkpoints/mobilenet_v2_qnnpack_37f702c5.pth


100%|██████████| 3.42M/3.42M [00:00<00:00, 16.9MB/s]

Loaded quantized INT8 MobileNetV2 from torchvision.models.quantization.
Quantized output shape: torch.Size([1, 1000])



/home/tokyog/edge_bootcamp/lib/python3.11/site-packages/torch/_utils.py:444: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


## 6. Benchmark Quantized INT8 MobileNetV2

We repeat the same measurements:

- Model size (MB)
- Latency (ms)
- RSS memory (MB)

> If the previous cell failed (import error), skip this or fix your environment.


In [6]:
if 'quantized_model' in globals():
    process = psutil.Process(os.getpid())
    
    int8_size_mb = get_model_size_mb(quantized_model, filename="mobilenet_v2_int8.pth")
    int8_latency_ms = measure_latency(quantized_model, example_input, n_warmup=5, n_iters=20)
    
    int8_rss_mb = process.memory_info().rss / (1024 * 1024)
    
    print(f"INT8 MobileNetV2 size: {int8_size_mb:.2f} MB")
    print(f"INT8 MobileNetV2 avg latency: {int8_latency_ms:.2f} ms")
    print(f"Approx. RSS memory (process): {int8_rss_mb:.2f} MB")
else:
    print("Quantized model is not available in this environment.")

INT8 MobileNetV2 size: 3.46 MB
INT8 MobileNetV2 avg latency: 18.09 ms
Approx. RSS memory (process): 527.11 MB


## 7. Compare Results

Fill in this table with your actual numbers from the outputs above:

| Model | Precision | Size (MB) | Latency (ms) | Speed-up vs FP32 | Notes |
|-------|-----------|-----------|--------------|------------------|-------|
| MobileNetV2 | FP32 | ⬜ | ⬜ | — | Baseline |
| MobileNetV2 | INT8 | ⬜ | ⬜ | ⬜ | Smaller & faster? |

> 💡 **Speed-up vs FP32** = `FP32_latency / INT8_latency`  
> Example: if FP32 = 40 ms and INT8 = 20 ms, speed-up = 2x.


## 8. Reflection Questions 🧠

Answer these in your own words (you can add a new Markdown cell below):

1. **Why does INT8 quantization usually reduce file size?**  
2. **How did latency change after quantization on your Raspberry Pi?**  
3. **If there is a small accuracy drop with INT8, why might that still be acceptable for edge devices?**  
4. **How could quantization help when deploying models on very small MCUs or when power usage is critical?**  

You can also try:

- Changing the number of iterations in `measure_latency`.
- Using a **larger batch size** in `example_input` (e.g., batch=4).
- Swapping MobileNetV2 for another model (e.g., ResNet18) and repeating the experiment.
